In [7]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import config
import logging
import importlib
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import validators
import pickle
import time
import random
from calendar import monthrange
import re
import spacy
from spacy.tokens import Doc, Span
import nltk
from nltk.corpus import stopwords
from nltk.corpus import treebank
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

from nltk import pos_tag, sent_tokenize, wordpunct_tokenize

pd.options.display.max_columns = 300
import string
import unicodedata

pd.options.display.max_columns = 200
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt  

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as datasets
import pandas as pd
import numpy as np
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
import seaborn as sns
import scipy.stats as stats
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.externals import joblib
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC  
from time import time
np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import LatentDirichletAllocation

import gensim
from gensim import corpora
from gensim import models
from gensim.sklearn_api import lsimodel, ldamodel
from gensim.matutils import sparse2full
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel

import pyLDAvis
import pyLDAvis.gensim

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from functools import partial

In [5]:
class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, language='english'):
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.more_stops = {'mr', 'ms', 'mr.', 'ms.', 'mrs', 'mrs.','say', 'also', 'yeh', 'hom'}
        self.stopwords.update(self.more_stops) 
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(unicodedata.category(char).startswith('P') for char in token)

    def is_stopword(self, token):
        return token.lower() in self.stopwords

    def get_wordnet_pos(selfm, word):
        #Map POS tag to first character lemmatize() 
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    def normalize(self, document, source):
        NYT_PAT = '^[A-Z]* — '
        NYT_COMP_PAT = re.compile(NYT_PAT)
        norm_toks = []

        if source == 'NYT':
            document = NYT_COMP_PAT.sub('', document)

        for sent in sent_tokenize(document):
            for tok in nltk.word_tokenize(sent):
                if self.is_stopword(tok) or self.is_punct(tok) or tok.isdigit():
                    continue
                norm_toks.append(self.lemmatize(tok, self.get_wordnet_pos(tok)).lower()) 
                
        return norm_toks


    def lemmatize(self, token, pos):
        return self.lemmatizer.lemmatize(token, pos)

    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        for doc in documents:
            yield self.normalize(document)

In [ ]:
nyt_df.dropna(inplace=True)

In [ ]:
txfm = partial(text_normalizer.normalize, source='NYT')

In [ ]:
nyt_df['norm_text'] = nyt_df.text.apply(txfm)

In [ ]:
lexicon = Dictionary([[word for word in doc] for doc in nyt_df.norm_text])

In [ ]:
corpus = [lexicon.doc2bow(doc) for doc in nyt_df.norm_text]

In [ ]:
corpus = [gmodel.model.named_steps['vect'].lexicon.doc2bow(doc),
            for doc in gmodel.model.named_steps['norm'].transform(docs)

In [ ]:
tfidf = TfidfModel([lexicon.doc2bow(doc) for doc in nyt_df.norm_text], id2word=lexicon)

In [ ]:
model = models.LdaModel(corpus, id2word=lexicon, num_topics=25)

In [ ]:
print(model.print_topics(10))

In [ ]:
id2token = lexicon.id2token

for word_id, freq in next(iter(corpus)):
    print(id2token[word_id], freq)
    # get the highest weighted topic for each of the documents in the corpus

def get_topics(vectorized_corpus, model):
    from operator import itemgetter
    topics = [
                max(model[doc], key=itemgetter(1))[0]
                for doc in vectorized_corpus]
    return topics

topics = get_topics(corpus, model)
for topic, doc in zip(topics, nyt_df.norm_text):
    print(\"Topic:{}\".format(topic))
    print(doc)

In [ ]:
data = pyLDAvis.gensim.prepare(model, corpus, lexicon)

In [ ]:
with open('gensim_data3.pkl', 'wb') as f:
    pickle.dump(data, f)
   

In [ ]:
pyLDAvis.display(data)

In [ ]:
pyLDAvis.save_html(data, 'gensim3.html')

In [ ]:
model.show_topic

In [ ]:
for t in range(model.num_topics):
    plt.figure()
    plt.imshow(WordCloud().fit_words(dict(model.show_topic(t, 200))))
    plt.axis(\"off\")
    plt.title(\"Topic #\" + str(t))
    plt.savefig(f'nyt_word_cloud_{t}.png')
    plt.show()\